In [1]:
import pandas as pd
import numpy as np
import os
import tarfile

In [7]:
data_path='J:/VeReMi-Dataset/MixAll_0024'
os.chdir(data_path)
datalist=os.listdir(data_path)

In [9]:
df=[]
with tarfile.open(datalist[0], "r:*") as tar:
    json_path = tar.getnames()[0]
    df.append(pd.read_json(tar.extractfile(json_path),lines=True))

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [ ]:


df=[]
for i in data:
    df.append(pd.read_json(i,lines=True))